In [ ]:
import mnist
import radiomics
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import cv2 as cv

In [ ]:
x_train = mnist.train_images()
y_train = mnist.train_labels()
x_test = mnist.test_images()
y_test = mnist.test_labels()

In [ ]:
def convert(samples):
    images = []
    for i, image in enumerate(samples):
        if i % 1000 == 0:
            print(i)

        _, im = cv.threshold(image, 128, 255, cv.THRESH_BINARY)
        mask = im.astype(np.float32) / 255.0
        mask = sitk.GetImageFromArray(mask)
        
        features = radiomics.shape2D.RadiomicsShape2D(sitk.GetImageFromArray(image), mask)
        features.enableAllFeatures()
        features.execute()
        features = np.array([v[1] for v in sorted(features.featureValues.items(), key=lambda x: x[0])])
        images.append(features)

    return np.stack(images)


np.save('mnist_train_radiomics.npy', convert(x_train))
np.save('mnist_test_radiomics.npy', convert(x_test))